In [1]:
import sys
import os
import ctypes
# newEnvPath = f"/fs/ssm/main/opt/intelcomp/inteloneapi-2022.1.2/intelcomp+mpi+mkl/all/lib/:{os.environ['LD_LIBRARY_PATH']}"
# libc = ctypes.cdll.LoadLibrary('libc.so.6')
# setenv = libc.setenv
# setenv.argtypes = (ctypes.c_char_p, ctypes.c_char_p, ctypes.c_int)
# setenv(b"LD_LIBRARY_PATH",newEnvPath.encode(), 1)
sys.path.append('/fs/site5/eccc/cmd/s/yor000/projects/librmn/localinstall/lib/python')
sys.path.append('/fs/site5/eccc/cmd/s/yor000/projects/georef/build/python/')

In [2]:
os.environ["LD_LIBRARY_PATH"]

'/.singularity.d/libs'

In [15]:
import georef
import xarray as xr
import fstd2nc
import rmn
from pathlib import Path

In [29]:
ECCI_DATA_DIR = Path('/home/sici000/ci_data')
input_arg = ECCI_DATA_DIR / 'libgeoref/grids/L.fstd'

In [31]:
str(input_arg)

'/home/sici000/ci_data/libgeoref/grids/L.fstd'

In [34]:
def copy_metadata(rec, template):
    rec.ni = template.ni
    rec.nj = template.nj
    rec.nk = template.nk

    rec.deet = template.deet
    rec.npas = template.npas

    rec.etiket = template.etiket
    rec.nomvar = template.nomvar
    rec.grtyp = template.grtyp
    rec.typvar = template.typvar

    rec.ip1 = template.ip1
    rec.ip2 = template.ip2
    rec.ip3 = template.ip3

    rec.ig1 = template.ig1
    rec.ig2 = template.ig2
    rec.ig3 = template.ig3
    rec.ig4 = template.ig4

    rec.dateo = template.dateo
    rec.datev = template.datev

def interpolate(input_file: str, output_file: str, truth: str, grid: str, vars: [str], etiket: str, interp_type: int):

    input_file = rmn.fst24_file(filename=input_file)

    output_file = rmn.fst24_file(filename=output_file, options='R/W')

    grid_file = rmn.fst24_file(filename=grid)

    options_1 = georef.GeoOptions(Interp=interp_type, NoData=0.0)


    grid_rec = next(grid_file.new_query(nomvar="GRID"))
    refout = georef.GeoRef(grid_rec.ni, grid_rec.nj, grid_rec.grtyp, grid_rec.ig1, grid_rec.ig2, grid_rec.ig3, grid_rec.ig4, grid_file)
    refout.valid()

    for rec_in in input_file.new_query(nomvar=vars[0]):
        refin = georef.GeoRef(rec_in.ni, rec_in.nj, rec_in.grtyp, rec_in.ig1, rec_in.ig2, rec_in.ig3, rec_in.ig4, input_file)
        refin.valid()

        georef.Interp(refout, refin, options_1, grid_rec.data, rec_in.data)
        copy_metadata(grid_rec, rec_in)
        output_file.write(grid_rec, rewrite=True)

In [35]:
interpolate(str(input_arg), 'output_file_yr.fstd', "skip", str(input_arg), ['GRID'], 'L2L', georef.RefInterpR.IR_LINEAR)

(ERROR) FST|c_xdfcheck: File size does not match header. Expected size: 208 bytes. Actual size: 18672
Before crash
After crash
Before crash
After crash


In [38]:
funit = rmn.fstopenall('/fs/site5/eccc/cmd/s/yor000/projects/georef/test/python/output_file.fstd')
xx_spi = rmn.fstlir(funit, nomvar='GRID', typvar='P')['d']
rmn.fstcloseall(funit)

funit = rmn.fstopenall('/fs/site5/eccc/cmd/s/yor000/projects/georef/test/python/output_file_yr.fstd')
xx_cs = rmn.fstlir(funit, nomvar='GRID', typvar='P@')['d']
rmn.fstcloseall(funit)

AttributeError: 'list' object has no attribute 'keys'